# Using CLMM on Real Datasets

_the LSST-DESC CLMM team_

This notebook can be run on NERSC.

Here we demonstrate how to run CLMM on real observational datasets. As an example, we use the data from the Dark Energy Survey (DES) public releases (DES Collaboration 2021, 2018). The catalogs can be accessed from the NOIRLab Astro Data Lab.

The steps in this notebook includes:
- [Setting things up](#Setup)
- [Selecting a cluster](#Selecting_a_cluster)
- [Downloading the published catalog at the cluster field](#Downloading_the_catalog)
- [Loading the catalog into CLMM](#Loading_the_catalog)
- [Running CLMM on the dataset](#Running_CLMM)

Acknowledgement

DES data: https://des.ncsa.illinois.edu/thanks

Astro Data Lab: https://datalab.noirlab.edu/acknowledgements.php


<a id="Setup"></a>
## 1. Setup
    
We import packages.

In [22]:
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
from astropy.table import Table
import pickle as pkl
from pathlib import Path

<a id="Selecting_a_cluster"></a>
## 2. Selecting a cluster

We use the DES Y1 redMaPPer Catalogs (https://des.ncsa.illinois.edu/releases/y1a1/key-catalogs/key-redmapper) to select a list of high-richness (LAMBDA) galaxy clusters, which likely have high masses.

Name | RA (deg) | DEC (deg) | Z_LAMBDA | LAMBDA | Note
- | - | - | - | - | -
RMJ025415.5-585710.7 | 43.564574 | -58.95297 | 0.429804 | 234.50368 | --
RMJ051637.4-543001.6 | 79.155704 | -54.500456 | 0.30416065 | 195.06956 | ACO S520
RMJ224851.8-443106.3 | 342.215897 | -44.518403 | 0.3514858 | 178.83827 | --
...





<a id="Downloading_the_catalog"></a>
## 3. Downloading the catalog at the cluster field

We consider RMJ051637.4-543001.6 (ACO S520) as an example.
We can access the DES DR1 catalog from NOIRLab Data Lab (https://datalab.noirlab.edu/query.php?name=des_dr1.shape_metacal_riz_unblind). No registration is required.
We make the query and download the catalogs in "Query Interface". 
We use `coadd_objects_id` to cross match the shape catalog, photo-z catalog, and photometry catalog. 
Since the cluster is at redshift about 0.3, a radius of 0.3 deg would be about a radial distance of 5 Mpc. 
The final catalog includes shape info, photo-z, and photometry. 
Here is an example of the query SQL command. 
The query could take a few minutes and the size of the catalog could be 3 MB (.csv). 

```
SELECT P.mean_z, 
M.ra, M.dec,
C.e1, C.e2, C.r11, C.r12, C.r21, C.r22 
FROM des_dr1.photo_z as P
INNER JOIN des_dr1.mof as M
ON P.coadd_objects_id=M.coadd_objects_id
INNER JOIN des_dr1.shape_metacal_riz_unblind as C
ON P.coadd_objects_id=C.coadd_objects_id
WHERE 't' = Q3C_RADIAL_QUERY(M.ra, M.dec,79.125000, -54.508333,0.3) 
--AND M.modest_class=1
--AND M.mof_flux_r/M.mof_fluxerr_r>5
--AND M.flags_r=0
--AND M.flags_gold=0
--AND M.mof_flags=0
AND P.minchi2<4
AND P.z_sigma<0.3
AND C.flags_select=0

```


<a id="Loading_the_catalog"></a>
## 4. Loading the catalog into CLMM

Once we have the catalog, we read in the catalog, make cuts on the catalog, and adjust column names to prepare for the analysis in CLMM.




In [23]:
%%time
# Assume the downloaded catalog is at this path:
filename = "../../../acos520.csv"
catalog = filename.replace('.csv', '.pkl')
if not Path(catalog).is_file():
    data_0 = Table.read(filename, format="ascii.csv")
    pkl.dump(data_0, open(catalog,"wb"))
else:
    data_0 = pkl.load(open(catalog,"rb"))

CPU times: user 5.42 ms, sys: 0 ns, total: 5.42 ms
Wall time: 4.43 ms


In [ ]:
print(data_0.colnames)

In [ ]:
data_1 = data_0

In [ ]:
# Mean of responsivity (to reduce noise).
# The diagonal terms are close.
# The off-diagonal terms are much smaller.
# We use the mean of the diagonal terms to reduce noise.
print(np.mean(data_1['r11']), np.mean(data_1['r22']))
print(np.mean(data_1['r12']), np.mean(data_1['r21']))
r_diag = np.mean([np.mean(data_1['r11']), np.mean(data_1['r22'])])
r_off_diag = np.mean([np.mean(data_1['r12']), np.mean(data_1['r21'])])
print(r_diag, r_off_diag, r_off_diag/r_diag)

In [ ]:
# Adjust column names.
def adjust_column_names(catalog_in):
    # We consider a map between new and old column names.
    # Note we have considered shear calibration here.
    column_name_map = {
        "ra": "ra",
        "dec": "dec",
        'z': "mean_z", 
        "e1": "e1",
        "e2": "e2",
    }
    
    catalog_out = Table()
    for i in column_name_map:
        catalog_out[i] = catalog_in[column_name_map[i]]
    
    catalog_out["e1"] /= r_diag
    catalog_out["e2"] /= r_diag
    
    return catalog_out

data_2 = adjust_column_names(data_1)

select = (data_2["e1"]**2 + data_2["e2"]**2 <=1.)
print(np.sum(~select) )
data_2 = data_2[select]

data_2["id"] = range(len(data_2))

In [ ]:
# Make some figures for visualization.
def make_plots(catalog_in):
    # Scatter plot
    plt.figure()
    plt.scatter(catalog_in["ra"], catalog_in["dec"], c=catalog_in['z'], s=1., alpha=0.2)
    plt.colorbar()
    plt.xlabel("ra")
    plt.ylabel("dec")
    plt.title("z")
    
    # Histogram
    plt.figure()
    plt.hist(catalog_in['z'], bins=20)
    plt.xlabel('z')
    plt.ylabel("count")
    
    # Relation
    plt.figure()
    plt.plot(catalog_in["e1"], catalog_in["e2"], ',')
    plt.xlabel("e1")
    plt.ylabel("e2")

make_plots(data_2)

<a id="Running_CLMM"></a>
## 5. Running CLMM on the dataset
We use the functions similar to `examples/Paper_v1.0/gt_and_use_case.ipynb`.

In [ ]:
from clmm import Cosmology
cosmo = Cosmology(H0=70.0, Omega_dm0=0.27-0.045, Omega_b0=0.045, Omega_k0=0.0)

# We consider RMJ051637.4-543001.6 (ACO S520)
cluster_z = 0.30416065 # Cluster redshift
cluster_ra = 79.155704 # Cluster Ra in deg
cluster_dec = -54.500456 # Cluster Dec in deg

obs_galaxies = data_2
 
obs_galaxies = obs_galaxies[(obs_galaxies['z']>(cluster_z+0.1))]

obs_galaxies['id'] = np.arange(len(obs_galaxies))

# Put galaxy values on arrays.
gal_ra = obs_galaxies['ra'] # Galaxies Ra in deg
gal_dec = obs_galaxies['dec'] # Galaxies Dec in deg
gal_e1 = obs_galaxies['e1'] # Galaxies elipticipy 1
gal_e2 = obs_galaxies['e2'] # Galaxies elipticipy 2
gal_z = obs_galaxies['z'] # Galaxies observed redshift
gal_id = obs_galaxies['id'] # Galaxies ID

In [ ]:
# Shear profile
# Using/testing clmm.dataops functions (details are in the notebook for Paper V1.0).
# This step is for testing the CLMM functionalities; we will not use the result in later steps.

import clmm.dataops as da
# Convert elipticities into shears.
gal_ang_dist, gal_gt, gal_gx = da.compute_tangential_and_cross_components(cluster_ra, cluster_dec,
                                                                          gal_ra, gal_dec,
                                                                          gal_e1, gal_e2,
                                                                          )

# Measure profile.
field_size = 6 # Mpc box size

profile = da.make_radial_profile([gal_gt, gal_gx, gal_z],
                                 gal_ang_dist, "radians", "Mpc",
                                 bins=da.make_bins(0.2, field_size/2., 10),
                                 cosmo=cosmo,
                                 z_lens=cluster_z,
                                 include_empty_bins=False)
print(f'Profile table has columns: {", ".join(profile.colnames)},')
print('where p_(0, 1, 2) = (gt, gx, z)')

In [ ]:
# Using the GalaxyCluster object.

import clmm
from clmm.utils import convert_units

# Create a GCData with the galaxies.
galaxies = clmm.GCData([gal_ra, gal_dec, gal_e1, gal_e2, gal_z, gal_id],
                      names=['ra', 'dec', 'e1', 'e2', 'z', 'id'])

# Create a GalaxyCluster.
cluster = clmm.GalaxyCluster("Name of cluster", cluster_ra, cluster_dec,
                                   cluster_z, galaxies)

# Convert elipticities into shears for the members.
cluster.compute_tangential_and_cross_components()
print(cluster.galcat.colnames)

# Measure profile and add profile table to the cluster. 
# This is just for showing the functionality.
seps = convert_units(cluster.galcat['theta'], 'radians', 'Mpc', cluster.z, cosmo)

cluster.make_radial_profile(bins=da.make_bins(0.4, 4.0, 7, method='evenlog10width'),
                            bin_units="Mpc",
                            cosmo=cosmo,
                            include_empty_bins=False,
                            gal_ids_in_bins=True,
                           )
print(cluster.profile.colnames)

In [ ]:
import sys
sys.path.append('Paper_v1.0')
from paper_formating import prep_plot
fig = prep_plot(figsize=(9, 9))
ax = fig.add_axes((0, 0, 1, 1))
errorbar_kwargs = dict(linestyle='', marker='o',
    markersize=1, elinewidth=.5, capthick=.5)
ax.errorbar(cluster.profile['radius'], cluster.profile['gt'],
             cluster.profile['gt_err'], c='k', **errorbar_kwargs)
ax.set_xlabel('r [Mpc]', fontsize = 10)
ax.set_ylabel(r'$g_t$', fontsize = 10)
plt.show()

In [ ]:
# Theoretical predictions

# Model relying on the overall redshift distribution of the sources (WtG III Applegate et al. 2014).

z_inf = 1000
dl_inf = cosmo.eval_da_z1z2(cluster_z, z_inf)
d_inf = cosmo.eval_da(z_inf)
concentration = 4.

def betas(z):
    dls = cosmo.eval_da_z1z2(cluster_z, z)
    ds = cosmo.eval_da(z)
    return dls * d_inf / (ds * dl_inf)

def predict_reduced_tangential_shear_redshift_distribution(profile, logm):

    bs_mean = np.mean(betas(cluster.galcat['z'])) 
    bs2_mean = np.mean(betas(cluster.galcat['z'])**2)

    gamma_t_inf = clmm.compute_tangential_shear(
            r_proj=profile['radius'], # Radial component of the profile
            mdelta=10**logm, # Mass of the cluster [M_sun]
            cdelta=concentration, # Concentration of the cluster
            z_cluster=cluster_z, # Redshift of the cluster
            z_source=z_inf, # Redshift value at infinity
            cosmo=cosmo,
            delta_mdef=200,
            massdef='critical', # For M200c
            halo_profile_model='nfw')
    convergence_inf = clmm.compute_convergence(
            r_proj=profile['radius'], # Radial component of the profile
            mdelta=10**logm, # Mass of the cluster [M_sun]
            cdelta=concentration, # Concentration of the cluster
            z_cluster=cluster_z, # Redshift of the cluster
            z_source=z_inf, # Redshift value at infinity
            cosmo=cosmo,
            delta_mdef=200,
            massdef='critical',
            halo_profile_model='nfw')
        
    return bs_mean*gamma_t_inf/(1-(bs2_mean/bs_mean)*convergence_inf)


# Model using individual redshift and radial information, to compute the averaged shear in each radial bin, based on the galaxies actually present in that bin.
cluster.galcat['theta_mpc'] = convert_units(cluster.galcat['theta'], 'radians', 'mpc',cluster.z, cosmo)

def predict_reduced_tangential_shear_individual_redshift(profile, logm):
    return np.array([np.mean(
        clmm.compute_reduced_tangential_shear(
            # Radial component of each source galaxy inside the radial bin
            r_proj=cluster.galcat[radial_bin['gal_id']]['theta_mpc'],
            mdelta=10**logm, # Mass of the cluster [M_sun]
            cdelta=concentration, # Concentration of the cluster
            z_cluster=cluster_z, # Redshift of the cluster
            # Redshift value of each source galaxy inside the radial bin
            z_source=cluster.galcat[radial_bin['gal_id']]['z'],
            cosmo=cosmo,
            delta_mdef=200,
            massdef='critical',
            halo_profile_model='nfw'
        )) for radial_bin in profile])


In [ ]:
# Mass fitting

mask_for_fit = cluster.profile['n_src'] > 2
data_for_fit = cluster.profile[mask_for_fit]

from clmm.support.sampler import fitters
def fit_mass(predict_function):
    popt, pcov = fitters['curve_fit'](predict_function,
        data_for_fit, 
        data_for_fit['gt'], 
        data_for_fit['gt_err'], bounds=[10.,17.])
    logm, logm_err = popt[0], np.sqrt(pcov[0][0])
    return {'logm':logm, 'logm_err':logm_err,
            'm': 10**logm, 'm_err': (10**logm)*logm_err*np.log(10)}

In [ ]:
%%time
fit_redshift_distribution = fit_mass(predict_reduced_tangential_shear_redshift_distribution)
fit_individual_redshift = fit_mass(predict_reduced_tangential_shear_individual_redshift)

In [ ]:
print(f'Best fit mass for N(z) model                     = {fit_redshift_distribution["m"]:.3e} +/- {fit_redshift_distribution["m_err"]:.3e} Msun')
print(f'Best fit mass for individual redshift and radius = {fit_individual_redshift["m"]:.3e} +/- {fit_individual_redshift["m_err"]:.3e} Msun')

In [ ]:
# Visualization of the results.
def get_predicted_shear(predict_function, fit_values):
    gt_est = predict_function(data_for_fit, fit_values['logm'])
    gt_est_err = [predict_function(data_for_fit, fit_values['logm']+i*fit_values['logm_err'])
                          for i in (-3, 3)]
    return gt_est, gt_est_err

gt_redshift_distribution, gt_err_redshift_distribution =  get_predicted_shear(predict_reduced_tangential_shear_redshift_distribution, fit_redshift_distribution)
gt_individual_redshift, gt_err_individual_redshift =  get_predicted_shear(predict_reduced_tangential_shear_individual_redshift, fit_individual_redshift)


In [ ]:
chi2_redshift_distribution_dof = np.sum((gt_redshift_distribution-data_for_fit['gt'])**2/(data_for_fit['gt_err'])**2)/(len(data_for_fit)-1)
chi2_individual_redshift_dof = np.sum((gt_individual_redshift-data_for_fit['gt'])**2/(data_for_fit['gt_err'])**2)/(len(data_for_fit)-1)

print(f'Reduced chi2 (N(z) model) = {chi2_redshift_distribution_dof}')
print(f'Reduced chi2 (individual (R,z) model) = {chi2_individual_redshift_dof}')

In [ ]:
from matplotlib.ticker import MultipleLocator
fig = prep_plot(figsize=(9 , 9))
gt_ax = fig.add_axes((.25, .42, .7, .55))
gt_ax.errorbar(data_for_fit['radius'],data_for_fit['gt'], data_for_fit['gt_err'],
             c='k',
            **errorbar_kwargs)

# Points in grey have not been used for the fit.
gt_ax.errorbar(cluster.profile['radius'][~mask_for_fit], cluster.profile['gt'][~mask_for_fit],
               cluster.profile['gt_err'][~mask_for_fit], 
               c='grey',**errorbar_kwargs)

pow10 = 14
mlabel = lambda name, fits: fr'$M_{{fit}}^{{{name}}} = {fits["m"]/10**pow10:.3f}\pm{fits["m_err"]/10**pow10:.3f}\times 10^{{{pow10}}} M_\odot$'

# The model for the 1st method.
gt_ax.loglog(data_for_fit['radius'], gt_redshift_distribution,'-C1', 
           label=mlabel('N(z)', fit_redshift_distribution),
             lw=.5)
gt_ax.fill_between(data_for_fit['radius'], *gt_err_redshift_distribution, lw=0, color='C1', alpha=.2)

# The model for the 2nd method.
gt_ax.loglog(data_for_fit['radius'], gt_individual_redshift,'-C2', 
           label=mlabel('z,R', fit_individual_redshift),
             lw=.5)
gt_ax.fill_between(data_for_fit['radius'], *gt_err_individual_redshift, lw=0, color='C2', alpha=.2)


gt_ax.set_ylabel(r'$g_t$', fontsize = 8)
gt_ax.legend(fontsize=6)
gt_ax.set_xticklabels([])
gt_ax.tick_params('x', labelsize=8)
gt_ax.tick_params('y', labelsize=8)


errorbar_kwargs2 = {k:v for k, v in errorbar_kwargs.items() if 'marker' not in k}
errorbar_kwargs2['markersize'] = 3
errorbar_kwargs2['markeredgewidth'] = .5
res_ax = fig.add_axes((.25, .2, .7, .2))
delta = (cluster.profile['radius'][1]/cluster.profile['radius'][0])**.25


res_ax.errorbar(data_for_fit['radius'], data_for_fit['gt']/gt_redshift_distribution-1,
                yerr=data_for_fit['gt_err']/gt_redshift_distribution, marker='s', c='C1', **errorbar_kwargs2)
errorbar_kwargs2['markersize'] = 3
errorbar_kwargs2['markeredgewidth'] = .5

res_ax.errorbar(data_for_fit['radius']*delta, data_for_fit['gt']/gt_individual_redshift-1,
                yerr=data_for_fit['gt_err']/gt_individual_redshift, marker='*', c='C2', **errorbar_kwargs2)
res_ax.set_xlabel(r'$R$ [Mpc]', fontsize = 8)

res_ax.set_ylabel(r'$g_t^{data}/g_t^{mod.}-1$', fontsize = 8)
res_ax.set_xscale('log')

res_ax.set_ylim(-5,5)
res_ax.yaxis.set_minor_locator(MultipleLocator(10))

res_ax.tick_params('x', labelsize=8)
res_ax.tick_params('y', labelsize=8)
plt.show()
